In [1]:
import numpy as np

<h1>Data reading methods</h1>

In [2]:
def padding_sequence(seq, max_len = 501, repkey = 'N'):
    seq_len = len(seq)
    if seq_len < max_len:
        gap_len = max_len -seq_len
        new_seq = seq + repkey * gap_len
    else:
        new_seq = seq[:max_len]
    return new_seq

In [3]:
def get_RNA_seq_concolutional_array(seq, motif_len = 4):
    seq = seq.replace('U', 'T')
    alpha = 'ACGT'
    #for seq in seqs:
    #for key, seq in seqs.iteritems():
    row = (len(seq) + 2*motif_len - 2)
    new_array = np.zeros((row, 4))
    for i in range(motif_len-1):
        new_array[i] = np.array([0.25]*4)
    
    for i in range(row-3, row):
        new_array[i] = np.array([0.25]*4)
        
    #pdb.set_trace()
    for i, val in enumerate(seq):
        i = i + motif_len-1
        # add .25's because this val is an N
        if val not in 'ACGT':
            new_array[i] = np.array([0.25]*4)
            continue
        #if val == 'N' or i < motif_len or i > len(seq) - motif_len:
        #    new_array[i] = np.array([0.25]*4)
        #else:
        try:
            index = alpha.index(val)
            new_array[i][index] = 1
        except:
            pdb.set_trace()
        #data[key] = new_array
    return new_array


In [4]:
def get_bag_data_1_channel(data, max_len = 501):
    bags = []
    seqs = data["seq"]
    labels = data["Y"]
    for seq in seqs:
        #pdb.set_trace()
        #bag_seqs = split_overlap_seq(seq)
        
        # replace each sequence with the sequence followed by N's until the length is equal to max_len
        bag_seq = padding_sequence(seq, max_len = max_len)
        #flat_array = []
        bag_subt = []
        #for bag_seq in bag_seqs:
        
        # turn the padded sequence into a 2-D array, where a row of the array is [.25,.25,.25,.25] for an N, and [1,0,0,0] for an A, [0,1,0,0] for an T, and so on
        tri_fea = get_RNA_seq_concolutional_array(bag_seq)
        bag_subt.append(tri_fea.T)

        
        bags.append(np.array(bag_subt))
    
        
    return bags, labels

In [5]:
def read_seq_graphprot(seq_file, label = 1):
    seq_list = []
    labels = []
    seq = ''
    with open(seq_file, 'r') as fp:
        for line in fp:
            if line[0] == '>':
                name = line[1:-1]
            else:
                seq = line[:-1].upper()
                seq = seq.replace('T', 'U')
                seq_list.append(seq)
                labels.append(label)
    
    return seq_list, labels

In [6]:
def read_data_file(posifile, negafile = None, train = True):
    data = dict()
    seqs, labels = read_seq_graphprot(posifile, label = 1)
    if negafile:
        seqs2, labels2 = read_seq_graphprot(negafile, label = 0)
        seqs = seqs + seqs2
        labels = labels + labels2
    # made a dictionary, "seq" key has a list of sequences as values, "Y" key has a list of 0's and 1's as values inidicating if the corresponding sequence is bound or not
    data["seq"] = seqs 
    data["Y"] = np.array(labels)
    
    return data

In [15]:
def get_data(posi, nega = None, channel = 1,  window_size = 501, train = True):
    data = read_data_file(posi, nega, train = train)
    if channel == 1:
        # pad the sequences and turn them into one-hot-encoded 2D arrays
        train_bags, label = get_bag_data_1_channel(data, max_len = window_size)

    else:
        train_bags, label = get_bag_data(data, channel = channel, window_size = window_size)
    
    return train_bags, label

<h1>Data exploration</h1>

In [8]:
# TASK 1: data exploration
train_data_dict = read_data_file('./CLIPSEQ_AGO2.train.positives.fa', './CLIPSEQ_AGO2.train.negatives.fa', train=True)
test_data_dict = read_data_file('./CLIPSEQ_AGO2.ls.positives.fa', './CLIPSEQ_AGO2.ls.negatives.fa', train=False)

In [9]:
print("The number of training samples is {} and the number of testing samples is {}".format(len(train_data_dict["seq"]), len(test_data_dict["seq"])))

sum = 0
max = 0
min = 1000
for seq in train_data_dict["seq"]:
    sum += len(seq)
    if len(seq) > max: max = len(seq)
    if len(seq) < min: min = len(seq)
avg_seq_len = sum/len(train_data_dict["seq"])

print("The average sequence length is {} basepairs with max length {} and min length {}".format(avg_seq_len, max, min))

pos = train_data_dict["Y"].sum()
pos_test = test_data_dict["Y"].sum()
print("The number of positive training samples is {}, negative {}, positive testing {}, negative testing {}".format(pos, len(train_data_dict["Y"])-pos, pos_test, len(test_data_dict["Y"])-pos_test ))                                                                                                         

The number of training samples is 92346 and the number of testing samples is 1000
The average sequence length is 335.3414333051783 basepairs with max length 375 and min length 137
The number of positive training samples is 48095, negative 44251, positive testing 500, negative testing 500


<h1>Read in data and format for NN</h1>

In [16]:
# read in the data 
f, l = get_data('./CLIPSEQ_AGO2.train.positives.fa', './CLIPSEQ_AGO2.train.negatives.fa', channel=1)

In [17]:
f_test, l_test = get_data('./CLIPSEQ_AGO2.ls.positives.fa','CLIPSEQ_AGO2.ls.negatives.fa', channel=1, train=False)

In [18]:
f = np.array(f)
f = np.swapaxes(f, -1, 1)

In [19]:
f_test = np.array(f_test)
f_test = np.swapaxes(f_test, -1, 1)

<h1>Model training</h1>

In [33]:
import keras
from keras import applications
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import SGD
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

<h3>Provided model from the github</h3>

In [28]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(10, 3),
                 activation='relu',
                 input_shape=f.shape[1:], padding='same'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(3, 1)))
model.add(Conv2D(128, (10, 1), activation='relu', padding='same'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(3, 1)))
model.add(Conv2D(256, (5, 1), activation='relu', padding='same'))
model.add(Dropout(0.25))
model.add(keras.layers.GlobalAveragePooling2D())
model.add(Dropout(0.25))
# model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [21]:
opt = SGD(lr=0.01)
# model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
model.compile(loss=keras.losses.binary_crossentropy, optimizer=opt,metrics=['accuracy'])

In [ ]:
model.fit(f, l,
          batch_size=10,
          epochs=5,
          verbose=1,
          validation_data=(f_test, l_test))

In [63]:
f.shape[1:]


(507, 4, 1)

<h3>Simple CNN Model</h3>

In [64]:
simple_model = Sequential()
# learning 128 different convolution filters => results 128 tensors output after this layer
# s
simple_model.add(Conv2D(128, kernel_size=(10, 4),
                 activation='relu',
                 input_shape=f.shape[1:], padding='same'))
simple_model.add(Dropout(0.25))
simple_model.add(MaxPooling2D(pool_size=(4, 1)))
simple_model.add(Conv2D(128, (10, 1), activation='relu', padding='same'))
simple_model.add(Dropout(0.25))
simple_model.add(BatchNormalization())
simple_model.add(keras.layers.GlobalAveragePooling2D())
simple_model.add(Dense(128, activation='relu'))
simple_model.add(Dropout(0.5))
simple_model.add(Dense(1, activation='sigmoid'))
simple_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 507, 4, 128)       5248      
_________________________________________________________________
dropout_24 (Dropout)         (None, 507, 4, 128)       0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 126, 4, 128)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 126, 4, 128)       163968    
_________________________________________________________________
dropout_25 (Dropout)         (None, 126, 4, 128)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 126, 4, 128)       512       
_________________________________________________________________
global_average_pooling2d_6 ( (None, 128)              

In [65]:
opt = SGD(lr=0.01)
# model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
simple_model.compile(loss=keras.losses.binary_crossentropy, optimizer=opt,metrics=['accuracy'])

In [66]:
f.shape

(92346, 507, 4, 1)

In [67]:
# subset the training data
to_del = [i for i in range(20000,92346-20000)]
f_sub = np.delete(f, to_del, 0)
l_sub = np.delete(l, to_del, 0)
l_sub.shape

(40000,)

In [ ]:
simple_model.fit(f, l,
          batch_size=10,
          epochs=50,
          verbose=1,
          validation_data=(f_test, l_test))

<h3> ResNet Model </h3>

<h1>Baselines</h1>

In [55]:
# hamming and edit distance baseline
from tinyalign import edit_distance, hamming_distance

In [59]:
# edit distance baseline
# done on raw, unpadded reads => length differences count toward edit distance
correct_predictions = 0
n = 1
for seq, label in zip (test_data_dict["seq"], test_data_dict["Y"]):
    min_dist = 1000
    # calculate edit distance between test seq and each of the training sequences (only use 10,000 for computation time)
    for train_seq, train_label in zip( train_data_dict["seq"][:10000],  train_data_dict["Y"][:10000] ):
        dist = edit_distance(seq, train_seq)
        if dist < min_dist: 
            min_dist = dist
            min_dist_label = train_label
    # will add one for each correct prediciton
    if min_dist_label == label:
        correct_predictions += 1
    if n%100 == 0:
        print("Percent done: {}%".format(n/1000))
    n+=1

print("edit distance baseline validation accuracy {}".format(correct_predictions/len(test_data_dict["Y"])))
    

Percent done: 0.1%
Percent done: 0.2%
Percent done: 0.3%
Percent done: 0.4%
Percent done: 0.5%
Percent done: 0.6%
Percent done: 0.7%
Percent done: 0.8%
Percent done: 0.9%
Percent done: 1.0%
edit distance baseline validation accuracy 0.5


In [61]:
# pad all the sequences
padded_test_data_dict = {}
padded_test_data_dict["Y"] = test_data_dict["Y"]
padded_train_data_dict = {}
padded_train_data_dict["Y"] = train_data_dict["Y"]

padded_test = []
for seq in test_data_dict["seq"]:
    padded_test.append(padding_sequence(seq))
padded_test_data_dict["seq"] = padded_test

padded_train = []
for seq in train_data_dict["seq"]:
    padded_train.append(padding_sequence(seq))
padded_train_data_dict["seq"] = padded_train

In [63]:
# hamming distance baseline
# this is done on the reads once they have already been padded with N's, so they are all the same length
correct_predictions = 0
n = 1
for seq, label in zip (padded_test_data_dict["seq"], padded_test_data_dict["Y"]):
    min_dist = 1000
    # calculate edit distance between test seq and each of the training sequences 
    for train_seq, train_label in zip(padded_train_data_dict["seq"], padded_train_data_dict["Y"]):
        dist = hamming_distance(seq, train_seq)
        if dist < min_dist:
            min_dist = dist
            min_dist_label = train_label
    # will add one for each correct prediciton
    if min_dist_label == label:
        correct_predictions += 1
    if n%100 == 0:
        print("Percent done: {}%".format(n/1000))
    n+=1

print("hamming distance baseline validation accuracy {}".format(correct_predictions/len(test_data_dict["Y"])))

Percent done: 0.1%
Percent done: 0.2%
Percent done: 0.3%
Percent done: 0.4%
Percent done: 0.5%
Percent done: 0.6%
Percent done: 0.7%
Percent done: 0.8%
Percent done: 0.9%
Percent done: 1.0%
hamming distance baseline validation accuracy 0.635


<a>edit distance baseline validation accuracy 0.5</a>
<a>hamming distance baseline validation accuracy 0.635</a>